<a href="https://colab.research.google.com/github/hazelhkim/Pytorch/blob/master/Basic_Model_for_Train_Valid_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics

In [3]:
BATCH_SIZE = 32

# transofrmatinos
transform = transforms.Compose([transforms.ToTensor()])

# Download and load training dataset
trainset = torchvision.datasets.MNIST(root = './data', train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers = 2)

# Download and load testing dataset
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [4]:
print(len(trainset)) #6000
print(len(trainset[0])) #2 data and label?
print(len(trainset[0][0])) #1
print(len(trainset[0][0][0])) #28
print(len(trainset[0][0][0][0])) #28

60000
2
1
28
28


In [5]:
class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()

    self.conv1 = nn.Conv2d(in_channels =1, out_channels=32, kernel_size=3)
    self.d1 = nn.Linear(26*26*32, 128)
    self.d2 = nn.Linear(128, 10)

  def forward(self, x):
    #32*1*28*28 => 32*32*26*26
    x = self.conv1(x)
    x = F.relu(x)

    #Flatten => 32 * (32*26*26)
    x = x.flatten(start_dim = 1)
    #x = x.view(32, -1)

    #32 *(32*26*26)=> 32*128
    x = self.d1(x)
    x = F.relu(x)

    #logits => 32*10
    logits = self.d2(x)
    out = F.softmax(logits, dim=1)
    return out

In [6]:
import numpy as np
a = np.array([[1,2],[3,4]])
b = np.ones((2,2))

ta = torch.tensor(a, dtype=float).to('cuda:0')
tb = torch.ones(2,2, dtype=float).to('cuda:0')

print(ta)
print(ta @ tb)

tensor([[1., 2.],
        [3., 4.]], device='cuda:0', dtype=torch.float64)
tensor([[3., 3.],
        [7., 7.]], device='cuda:0', dtype=torch.float64)


In [0]:
learning_rate = 0.001
num_epochs = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MyModel()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [8]:
for epoch in range(num_epochs):
  train_running_loss = 0.0
  train_acc = 0.0

  # training step
  for i, (images, labels) in enumerate(trainloader):
    images = images.to(device)
    labels = labels.to(device)

    #forward + backprop + loss
    logits = model(images)
    loss = criterion(logits, labels)
    optimizer.zero_grad()
    loss.backward()

    #update model params
    optimizer.step()

    train_running_loss += loss.detach().item()
    train_acc += (torch.argmax(logits, 1).flatten() == labels).type(torch.float).mean().item()

print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
      %(epoch, train_running_loss/i, train_acc/i) )

Epoch: 4 | Loss: 1.4750 | Train Accuracy: 0.99


In [9]:
test_acc = 0.0
for i, (images, labels) in enumerate(testloader, 0):
  images = images.to(device)
  labels = labels.to(device)
  outputs = model(images)
  test_acc += (torch.argmax(outputs,1).flatten()==labels).type(torch.float).mean().item()
  preds = torch.argmax(outputs, 1).flatten().cpu().numpy()

print('Test Accuracy %.2f' %(test_acc/i))

Test Accuracy 0.99
